In [850]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import tensorflow as tf
import sklearn
from prettytable import PrettyTable

In [851]:
data = pd.read_csv('resources/EUR_USD_2010-2019_c5min_10_pred15min.csv', index_col=0)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,up,down
0,0.000000,0.064327,0.225133,0.132626,0.359389,0.943263,0.678927,0.094164,0.110546,0.659689,...,0.000000,0.419183,0.263994,0.157825,0.055193,0.844248,0.418667,0.061008,1.0,0.0
1,0.359389,0.943263,0.678927,0.094164,0.110546,0.659689,0.415038,0.090186,0.117471,0.836911,...,0.055193,0.844248,0.418667,0.061008,0.407119,0.879626,0.650530,0.094164,0.0,1.0
2,0.110546,0.659689,0.415038,0.090186,0.117471,0.836911,0.415026,0.054377,0.635815,1.000000,...,0.407119,0.879626,0.650530,0.094164,0.689811,0.971686,0.797990,0.155172,0.0,1.0
3,0.117471,0.836911,0.415026,0.054377,0.635815,1.000000,0.819669,0.049072,0.041440,0.978738,...,0.689811,0.971686,0.797990,0.155172,0.082726,0.278432,0.137927,0.156499,1.0,0.0
4,0.635815,1.000000,0.819669,0.049072,0.041440,0.978738,0.510012,0.015915,0.531796,1.000000,...,0.082726,0.278432,0.137927,0.156499,0.068986,0.730997,0.411649,0.128647,1.0,0.0


In [852]:
values = data.drop(['up', 'down'], 1)
labels = data[['up', 'down']]

In [853]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(values.values, labels.values, test_size=0.2, shuffle=True, random_state=111)

In [854]:
from tensorflow.python.keras.layers import Dense, Dropout

def create_model(input_dim, act, dropout, layers):
    model = tf.keras.Sequential()
    if layers == 1:
        model.add(Dense(input_dim, input_dim=input_dim, activation=act))
        model.add(Dropout(dropout))
    if layers == 2:
        model.add(Dense(input_dim, input_dim=input_dim, activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.5), activation=act))
        model.add(Dropout(dropout))
    if layers == 3:
        model.add(Dense(input_dim, input_dim=input_dim, activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.5), activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.3), activation=act))
        model.add(Dropout(dropout))
    if layers == 4:
        model.add(Dense(input_dim, input_dim=input_dim, activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.7), activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.5), activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.3), activation=act))
        model.add(Dropout(dropout))
    if layers == 5:
        model.add(Dense(input_dim, input_dim=input_dim, activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.8), activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.6), activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.4), activation=act))
        model.add(Dropout(dropout))
        model.add(Dense(int(input_dim * 0.2), activation=act))
        model.add(Dropout(dropout))
    model.add(Dense(2, activation="softmax"))
    model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
    return model

def test_result(model, x_test, y_test):
    n = x_test.shape[0]
    a = np.empty(shape=[n, 1])
    pred = model.predict(x_test)
    table = PrettyTable()
    best_res = 0
    table.field_names = ['Answer', 'Win', 'Lose', 'Win (%)', 'Answer (%)']
    for i, pr in enumerate(pred): a[i] = int(round((pr[0] - pr[1]) * 100, 0))
    results = np.hstack([a, y_test])
    for lim in range(0, 99, 2):
        win = lose = 0
        for i, d in enumerate(results):
            if abs(d[0]) >= lim:
                if (d[0] > 0 and d[1] == 1) or (d[0] < 0 and d[1] == 0):
                    win += 1
                else:
                    lose += 1
        if win + lose > 0:
            res = int((win / (win + lose)) * 100)
        else:
            res = 0
        trades = round(((win + lose) / results.shape[0]) * 100, 1)
        if trades < 1: break
        # best_res = res
        # best_lim = lim
        # best_trades = trades
        table.add_row([lim, win, lose, res, trades])
    # if best_res <= n_lim:
    #     clear_output(wait=True)
    #     print("⚠️️ Результат <", n_lim, ". Очень низкое качество обучения." +
    #           "\nПовторите обучение c другими настройками нейросети.")
    #     print("Заново загружать данные не нужно.")
    # else:
    print(table)
    # print("\n✅️ Последнее значение -", str(best_res) + "% правильных ответов.\n" +
    #         'Этот результат будет получен если брать ответы с "уверенность" больше', best_lim, "(Ответ сети).",
    #         "\nСигналов будет -", str(best_trades) +
    #         "% от общего количества свечей которые подавались на тестирование.")
    # return best_lim, best_res, results

In [855]:
act = 'tanh'
dropout = 0.2
epochs = 300
input_dim = values.shape[1]

model = create_model(input_dim, act, dropout, 4)
history = model.fit(x_train, y_train, epochs=epochs, validation_split=0.2, batch_size=128, verbose=1)

Epoch 1/300
320/320 [==============================] - 0s 1ms/step - loss: 0.7181 - accuracy: 0.5055 - val_loss: 0.6931 - val_accuracy: 0.5060
Epoch 2/300
320/320 [==============================] - 0s 874us/step - loss: 0.6970 - accuracy: 0.5045 - val_loss: 0.6928 - val_accuracy: 0.5131

In [856]:
test_result(model, x_test, y_test)


+--------+------+------+---------+------------+
| Answer | Win  | Lose | Win (%) | Answer (%) |
+--------+------+------+---------+------------+
|   0    | 6276 | 6503 |    49   |   100.0    |
|   2    | 5792 | 5488 |    51   |    88.3    |
|   4    | 4931 | 4577 |    51   |    74.4    |
|   6    | 4096 | 3808 |    51   |    61.9    |
|   8    | 3381 | 3132 |    51   |    51.0    |
|   10   | 2763 | 2545 |    52   |    41.5    |
|   12   | 2118 | 1951 |    52   |    31.8    |
|   14   | 1494 | 1343 |    52   |    22.2    |
|   16   | 901  | 806  |    52   |    13.4    |
|   18   | 372  | 348  |    51   |    5.6     |
|   20   | 103  |  99  |    50   |    1.6     |
+--------+------+------+---------+------------+
